# Chapter 14: Introduction to LangGraph - Solutions
**From: Zero to AI Agent**

**Try the exercises in the main notebook first before viewing solutions!**

---
## Section 14.1 Solutions

### Exercise 14.1.1: Identify Chain Limitations

In [ ]:
# Solution file not found: exercise_1_14_1_solution.py

### Exercise 14.1.2: Flowchart Design

In [ ]:
# Solution file not found: exercise_2_14_1_solution.py

### Exercise 14.1.3: Analyze the Pattern

In [ ]:
# Solution file not found: exercise_3_14_1_solution.py

---
## Section 14.2 Solutions

### Exercise 14.2.1: Pattern Recognition

In [ ]:
# Solution file not found: exercise_1_14_2_solution.py

### Exercise 14.2.2: Design a Recipe Agent

In [ ]:
# Solution file not found: exercise_2_14_2_solution.py

### Exercise 14.2.3: Identify the State

In [ ]:
# Solution file not found: exercise_3_14_2_solution.py

---
## Section 14.3 Solutions

### Exercise 14.3.1: Environment Exploration

In [ ]:
# Solution file not found: exercise_1_14_3_solution.py

### Exercise 14.3.2: API Key Security

In [ ]:
# Solution file not found: exercise_2_14_3_solution.py

### Exercise 14.3.3: Create a Setup Checker

In [ ]:
# File: exercise_3_14_3_solution.py (Comprehensive Setup Checker)

"""Comprehensive setup verification for LangGraph development."""

import sys

def print_header(title):
    """Print a formatted section header."""
    print(f"\n{'='*50}")
    print(f"  {title}")
    print('='*50)

def check_python_version():
    """Check Python version is 3.9+."""
    version = sys.version_info
    if version.major >= 3 and version.minor >= 9:
        print(f"✅ Python {version.major}.{version.minor}.{version.micro}")
        return True
    else:
        print(f"❌ Python {version.major}.{version.minor} (need 3.9+)")
        return False

def check_packages():
    """Check all required packages are installed."""
    packages = {
        'langgraph': 'langgraph',
        'langchain': 'langchain', 
        'langchain_openai': 'langchain-openai',
        'dotenv': 'python-dotenv'
    }
    
    all_good = True
    for import_name, package_name in packages.items():
        try:
            __import__(import_name)
            print(f"✅ {package_name}")
        except ImportError:
            print(f"❌ {package_name} - run: pip install {package_name}")
            all_good = False
    
    return all_good

def check_api_key():
    """Check API key is configured."""
    import os
    from dotenv import load_dotenv
    
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    
    if not api_key:
        print("❌ OPENAI_API_KEY not found")
        print("   Create a .env file with: OPENAI_API_KEY=sk-...")
        return False
    
    if not api_key.startswith("sk-"):
        print("⚠️  API key format looks wrong (should start with 'sk-')")
        return False
    
    # Mask the key for display
    masked = api_key[:7] + "..." + api_key[-4:]
    print(f"✅ API key found ({masked})")
    return True

def check_api_connection():
    """Test actual API connection."""
    try:
        from langchain_openai import ChatOpenAI
        
        print("🔄 Testing API connection...")
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=10)
        response = llm.invoke("Say 'OK'")
        print(f"✅ API connection working")
        return True
        
    except Exception as e:
        error_msg = str(e)
        if "authentication" in error_msg.lower():
            print("❌ API authentication failed - check your key")
        elif "rate" in error_msg.lower():
            print("❌ Rate limited - wait a minute and try again")
        elif "quota" in error_msg.lower():
            print("❌ No API quota - add credits to your OpenAI account")
        else:
            print(f"❌ API error: {error_msg[:100]}")
        return False

def check_langgraph_imports():
    """Check LangGraph components are importable."""
    try:
        from langgraph.graph import StateGraph, END
        from langgraph.checkpoint.memory import MemorySaver
        print("✅ LangGraph components accessible")
        return True
    except ImportError as e:
        print(f"❌ LangGraph import failed: {e}")
        return False

def main():
    """Run all checks and report results."""
    print("\n🔍 LangGraph Setup Checker")
    print("=" * 50)
    
    results = {}
    
    # Check Python
    print_header("Python Version")
    results['python'] = check_python_version()
    
    # Check packages
    print_header("Required Packages")
    results['packages'] = check_packages()
    
    # Check LangGraph imports
    print_header("LangGraph Components")
    results['langgraph'] = check_langgraph_imports()
    
    # Check API key
    print_header("API Configuration")
    results['api_key'] = check_api_key()
    
    # Check API connection (only if key exists)
    if results['api_key']:
        print_header("API Connection Test")
        results['api_connection'] = check_api_connection()
    else:
        results['api_connection'] = False
    
    # Summary
    print_header("Summary")
    
    all_passed = all(results.values())
    passed = sum(results.values())
    total = len(results)
    
    for check, result in results.items():
        status = "✅" if result else "❌"
        print(f"  {status} {check.replace('_', ' ').title()}")
    
    print(f"\n  Result: {passed}/{total} checks passed")
    
    if all_passed:
        print("\n🎉 All checks passed! You're ready to build with LangGraph!")
    else:
        print("\n⚠️  Some checks failed. Fix the issues above and run again.")
        print("   Need help? Check the troubleshooting section in 14.3")
    
    return all_passed

if __name__ == "__main__":
    success = main()
    sys.exit(0 if success else 1)


---
## Section 14.4 Solutions

### Exercise 14.4.1: Design a State

In [ ]:
# Solution file not found: exercise_1_14_4_solution.py

### Exercise 14.4.2: Write the Nodes

In [ ]:
# Solution file not found: exercise_2_14_4_solution.py

### Exercise 14.4.3: Draw the Graph

In [ ]:
# File: exercise_3_14_4_solution.py (Code Review Agent - combines exercises 1, 2, 3)

"""
Complete Code Review Agent demonstrating:
- Exercise 1: State design with TypedDict
- Exercise 2: Node implementations
- Exercise 3: Graph construction with conditional edges

This agent analyzes code, identifies issues, suggests fixes for each,
and loops until all issues are addressed.
"""

from typing import TypedDict, Annotated, Optional
from operator import add
from langgraph.graph import StateGraph, END


# =============================================================================
# EXERCISE 1 SOLUTION: State Design
# =============================================================================

class CodeReviewState(TypedDict):
    # Input
    code: str                                    # The code to review
    language: str                                # Programming language
    
    # Analysis results
    issues: Annotated[list, add]                 # List of identified issues
                                                 # Each issue: {"id": str, "severity": str, 
                                                 #              "description": str, "line": int}
    
    # Fix tracking  
    suggested_fixes: Annotated[list, add]        # Fixes for issues
                                                 # Each fix: {"issue_id": str, "suggestion": str,
                                                 #            "fixed_code": str}
    
    addressed_issue_ids: Annotated[list, add]    # IDs of issues that have fixes
    
    # Control flow
    current_issue_index: int                     # Which issue we're working on
    review_complete: bool                        # Are we done?
    
    # Optional metadata
    summary: Optional[str]                       # Final review summary


# =============================================================================
# EXERCISE 2 SOLUTION: Node Implementations
# =============================================================================

def analyze_code(state: CodeReviewState) -> dict:
    """Analyze the code and identify issues."""
    code = state["code"]
    language = state["language"]
    
    # In reality, this would use an LLM or static analysis tool
    # Pseudocode for the logic:
    #
    # prompt = f"""Analyze this {language} code for issues:
    # {code}
    # 
    # Return a list of issues with severity (high/medium/low),
    # description, and line number."""
    # 
    # response = llm.invoke(prompt)
    # issues = parse_issues(response)
    
    # For demo, pretend we found some issues:
    issues = [
        {"id": "issue_1", "severity": "high", 
         "description": "Potential null pointer", "line": 15},
        {"id": "issue_2", "severity": "medium",
         "description": "Unused variable", "line": 8},
    ]
    
    return {
        "issues": issues,
        "current_issue_index": 0,  # Start with first issue
        "review_complete": False
    }


def suggest_fix(state: CodeReviewState) -> dict:
    """Suggest a fix for the current issue."""
    issues = state["issues"]
    current_index = state["current_issue_index"]
    code = state["code"]
    
    # Get the current issue
    current_issue = issues[current_index]
    
    # In reality, this would use an LLM:
    # prompt = f"""Given this code:
    # {code}
    # 
    # Suggest a fix for this issue:
    # {current_issue['description']} on line {current_issue['line']}
    # 
    # Return the suggested fix and corrected code snippet."""
    #
    # response = llm.invoke(prompt)
    # fix = parse_fix(response)
    
    # For demo:
    fix = {
        "issue_id": current_issue["id"],
        "suggestion": f"Fix for {current_issue['description']}",
        "fixed_code": "# corrected code here"
    }
    
    return {
        "suggested_fixes": [fix],
        "addressed_issue_ids": [current_issue["id"]],
        "current_issue_index": current_index + 1  # Move to next issue
    }


def check_complete(state: CodeReviewState) -> dict:
    """Check if all issues have been addressed."""
    issues = state["issues"]
    addressed_ids = state["addressed_issue_ids"]
    
    # Are all issues addressed?
    all_issue_ids = {issue["id"] for issue in issues}
    addressed_set = set(addressed_ids)
    
    is_complete = all_issue_ids == addressed_set
    
    # If complete, generate summary
    if is_complete:
        summary = f"Review complete. Found {len(issues)} issues, all addressed."
    else:
        summary = None  # Don't set summary until complete
    
    return {
        "review_complete": is_complete,
        "summary": summary if is_complete else state.get("summary")
    }


# =============================================================================
# EXERCISE 3 SOLUTION: Graph Construction
# =============================================================================

def should_continue(state: CodeReviewState) -> str:
    """Decide whether to continue or finish."""
    if state["review_complete"]:
        return "done"
    else:
        return "continue"


def build_code_review_graph():
    """Build and return the code review agent graph."""
    
    # Create the graph with our state type
    graph = StateGraph(CodeReviewState)
    
    # Add all nodes
    graph.add_node("analyze_code", analyze_code)
    graph.add_node("suggest_fix", suggest_fix)
    graph.add_node("check_complete", check_complete)
    
    # Set the entry point
    graph.set_entry_point("analyze_code")
    
    # Add edges
    graph.add_edge("analyze_code", "suggest_fix")
    graph.add_edge("suggest_fix", "check_complete")
    
    # Conditional edge from check_complete
    graph.add_conditional_edges(
        "check_complete",
        should_continue,
        {
            "done": END,
            "continue": "suggest_fix"  # Loop back
        }
    )
    
    # Compile and return
    return graph.compile()


# =============================================================================
# Main execution
# =============================================================================

if __name__ == "__main__":
    # Build the graph
    app = build_code_review_graph()
    
    # Example code to review
    sample_code = """
def process_data(data):
    unused_var = 42
    result = data.get('value')
    return result.upper()  # Potential None error
"""
    
    # Run the review
    result = app.invoke({
        "code": sample_code,
        "language": "python",
        "issues": [],
        "suggested_fixes": [],
        "addressed_issue_ids": [],
        "current_issue_index": 0,
        "review_complete": False
    })
    
    # Display results
    print("=" * 50)
    print("CODE REVIEW RESULTS")
    print("=" * 50)
    
    print(f"\n📋 Issues Found: {len(result['issues'])}")
    for issue in result['issues']:
        print(f"   [{issue['severity'].upper()}] Line {issue['line']}: {issue['description']}")
    
    print(f"\n🔧 Fixes Suggested: {len(result['suggested_fixes'])}")
    for fix in result['suggested_fixes']:
        print(f"   Issue {fix['issue_id']}: {fix['suggestion']}")
    
    print(f"\n✅ {result['summary']}")


---
## Section 14.5 Solutions

### Exercise 14.5.1: Add Draft History

In [ ]:
# File: exercise_1_14_5_solution.py (Writer with Draft History)

"""Self-improving writer that keeps history of all drafts.

Exercise 1: Modify the writer to keep a history of all drafts using
Annotated[list, add] so you can see how the writing evolved.
"""

import os
from typing import TypedDict, Annotated
from operator import add
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


class WriterState(TypedDict):
    topic: str
    drafts: Annotated[list, add]     # History of all drafts
    current_draft: str                # Most recent draft
    critique: str
    revision_count: int
    max_revisions: int


def write_draft(state: WriterState) -> dict:
    """Write the initial draft."""
    topic = state["topic"]
    
    prompt = f"""Write a short, informative paragraph about: {topic}
    Keep it concise but engaging. Aim for 3-4 sentences."""
    
    response = llm.invoke(prompt)
    draft = response.content
    
    print(f"📝 Draft written ({len(draft)} chars)")
    
    return {
        "current_draft": draft,
        "drafts": [{"version": 1, "content": draft}],  # Appends to list
        "revision_count": 0
    }


def critique_draft(state: WriterState) -> dict:
    """Critique the current draft."""
    draft = state["current_draft"]
    topic = state["topic"]
    
    prompt = f"""Review this draft about "{topic}" and provide brief feedback.
    
    Draft:
    {draft}
    
    If excellent, say "EXCELLENT" at the start. Otherwise give 2-3 suggestions."""
    
    response = llm.invoke(prompt)
    print(f"🔍 Critique provided")
    
    return {"critique": response.content}


def revise_draft(state: WriterState) -> dict:
    """Revise based on feedback."""
    draft = state["current_draft"]
    critique = state["critique"]
    topic = state["topic"]
    revision_count = state["revision_count"]
    
    prompt = f"""Revise this draft about "{topic}" based on feedback:
    
    Current draft: {draft}
    Feedback: {critique}
    
    Write an improved version."""
    
    response = llm.invoke(prompt)
    new_draft = response.content
    new_count = revision_count + 1
    
    print(f"✏️ Revision {new_count} complete")
    
    return {
        "current_draft": new_draft,
        "drafts": [{"version": new_count + 1, "content": new_draft}],  # Appends
        "revision_count": new_count
    }


def should_continue(state: WriterState) -> str:
    """Decide whether to continue revising."""
    if state["revision_count"] >= state["max_revisions"]:
        return "end"
    if "EXCELLENT" in state["critique"].upper():
        return "end"
    return "revise"


def create_graph():
    graph = StateGraph(WriterState)
    
    graph.add_node("write_draft", write_draft)
    graph.add_node("critique", critique_draft)
    graph.add_node("revise", revise_draft)
    
    graph.set_entry_point("write_draft")
    graph.add_edge("write_draft", "critique")
    graph.add_conditional_edges("critique", should_continue, {"revise": "revise", "end": END})
    graph.add_edge("revise", "critique")
    
    return graph.compile()


def main():
    app = create_graph()
    
    result = app.invoke({
        "topic": "The benefits of reading books",
        "drafts": [],
        "current_draft": "",
        "critique": "",
        "revision_count": 0,
        "max_revisions": 3
    })
    
    # Display all versions
    print("\n" + "=" * 50)
    print("📚 DRAFT HISTORY:")
    print("=" * 50)
    
    for draft in result["drafts"]:
        print(f"\n--- Version {draft['version']} ---")
        print(draft["content"])
    
    print("\n" + "=" * 50)
    print(f"Total versions: {len(result['drafts'])}")


if __name__ == "__main__":
    main()


### Exercise 14.5.2: Quality Scoring

In [ ]:
# File: exercise_2_14_5_solution.py (Writer with Quality Scoring)

"""Self-improving writer with quality scoring.

Exercise 2: Add a numeric quality score (1-10) to the process.
The writer continues until the score reaches 8 or higher.
"""

import os
import re
from typing import TypedDict
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


class WriterState(TypedDict):
    topic: str
    draft: str
    critique: str
    quality_score: int               # Numeric quality score (1-10)
    score_history: list              # Track scores over time
    revision_count: int
    max_revisions: int


def write_draft(state: WriterState) -> dict:
    """Write the initial draft."""
    topic = state["topic"]
    
    prompt = f"""Write a short, informative paragraph about: {topic}
    Keep it concise but engaging. Aim for 3-4 sentences."""
    
    response = llm.invoke(prompt)
    
    print(f"📝 Draft written")
    
    return {
        "draft": response.content,
        "revision_count": 0,
        "quality_score": 0,
        "score_history": []
    }


def critique_draft(state: WriterState) -> dict:
    """Critique and score the draft."""
    draft = state["draft"]
    topic = state["topic"]
    
    prompt = f"""Review this draft about "{topic}".
    
    Draft:
    {draft}
    
    Provide:
    1. A quality score from 1-10 (format: "SCORE: X")
    2. Brief feedback for improvement
    
    Be a tough but fair critic. Only give 9-10 for truly excellent writing."""
    
    response = llm.invoke(prompt)
    critique_text = response.content
    
    # Extract score from response
    score_match = re.search(r'SCORE:\s*(\d+)', critique_text, re.IGNORECASE)
    score = int(score_match.group(1)) if score_match else 5
    score = max(1, min(10, score))  # Clamp to 1-10
    
    # Update score history
    new_history = state.get("score_history", []) + [score]
    
    print(f"🔍 Critique: Score {score}/10")
    
    return {
        "critique": critique_text,
        "quality_score": score,
        "score_history": new_history
    }


def revise_draft(state: WriterState) -> dict:
    """Revise based on feedback."""
    draft = state["draft"]
    critique = state["critique"]
    topic = state["topic"]
    revision_count = state["revision_count"]
    
    prompt = f"""Revise this draft about "{topic}" based on feedback:
    
    Current draft: {draft}
    Feedback: {critique}
    
    Write an improved version that addresses the feedback."""
    
    response = llm.invoke(prompt)
    
    new_count = revision_count + 1
    print(f"✏️ Revision {new_count} complete")
    
    return {
        "draft": response.content,
        "revision_count": new_count
    }


def should_continue(state: WriterState) -> str:
    """Decide based on score and revision count."""
    score = state["quality_score"]
    revision_count = state["revision_count"]
    max_revisions = state["max_revisions"]
    
    # Stop if score is 8 or higher
    if score >= 8:
        print(f"✨ Quality score {score}/10 - excellent!")
        return "end"
    
    # Stop if max revisions reached
    if revision_count >= max_revisions:
        print(f"🛑 Max revisions reached (score: {score}/10)")
        return "end"
    
    print(f"🔄 Score {score}/10 - continuing...")
    return "revise"


def create_graph():
    graph = StateGraph(WriterState)
    
    graph.add_node("write_draft", write_draft)
    graph.add_node("critique", critique_draft)
    graph.add_node("revise", revise_draft)
    
    graph.set_entry_point("write_draft")
    graph.add_edge("write_draft", "critique")
    graph.add_conditional_edges("critique", should_continue, {"revise": "revise", "end": END})
    graph.add_edge("revise", "critique")
    
    return graph.compile()


def main():
    app = create_graph()
    
    result = app.invoke({
        "topic": "The importance of sleep for health",
        "draft": "",
        "critique": "",
        "quality_score": 0,
        "score_history": [],
        "revision_count": 0,
        "max_revisions": 3
    })
    
    print("\n" + "=" * 50)
    print("📄 FINAL DRAFT:")
    print("=" * 50)
    print(result["draft"])
    
    print("\n" + "-" * 50)
    print(f"📊 Score progression: {' → '.join(map(str, result['score_history']))}")
    print(f"📊 Final score: {result['quality_score']}/10")
    print(f"📊 Total revisions: {result['revision_count']}")


if __name__ == "__main__":
    main()


### Exercise 14.5.3: Different Writing Styles

In [ ]:
# File: exercise_3_14_5_solution.py (Writer with Style Options)

"""Self-improving writer with style options.

Exercise 3: Add a style parameter (formal/casual/creative) that changes
how the writer creates and evaluates content.
"""

import os
from typing import TypedDict
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


# Style definitions
STYLES = {
    "formal": {
        "description": "professional, business-like, using industry terminology",
        "tone": "authoritative and polished",
        "example": "formal business report"
    },
    "casual": {
        "description": "friendly, conversational, approachable",
        "tone": "warm and relatable, like talking to a friend",
        "example": "blog post or social media"
    },
    "creative": {
        "description": "artistic, expressive, using vivid imagery",
        "tone": "imaginative and evocative",
        "example": "creative essay or storytelling"
    }
}


class WriterState(TypedDict):
    topic: str
    style: str                       # Writing style (formal/casual/creative)
    draft: str
    critique: str
    revision_count: int
    max_revisions: int


def get_style_prompt(style: str) -> str:
    """Get style instructions for prompts."""
    style_info = STYLES.get(style, STYLES["casual"])
    return f"""Style: {style_info['description']}
Tone: {style_info['tone']}
Write as if for: {style_info['example']}"""


def write_draft(state: WriterState) -> dict:
    """Write initial draft in the specified style."""
    topic = state["topic"]
    style = state["style"]
    style_instructions = get_style_prompt(style)
    
    prompt = f"""Write a short paragraph about: {topic}

{style_instructions}

Keep it to 3-4 sentences while maintaining the style throughout."""
    
    response = llm.invoke(prompt)
    
    print(f"📝 Draft written in {style} style")
    
    return {
        "draft": response.content,
        "revision_count": 0
    }


def critique_draft(state: WriterState) -> dict:
    """Critique with style considerations."""
    draft = state["draft"]
    topic = state["topic"]
    style = state["style"]
    style_info = STYLES.get(style, STYLES["casual"])
    
    prompt = f"""Review this {style} draft about "{topic}".
    
    Draft:
    {draft}
    
    The intended style is: {style_info['description']}
    The intended tone is: {style_info['tone']}
    
    Evaluate:
    1. Does it match the intended style and tone?
    2. Is the content accurate and engaging?
    3. What specific improvements would make it better?
    
    If it's excellent for the style, say "EXCELLENT" at the start."""
    
    response = llm.invoke(prompt)
    
    print(f"🔍 Critique for {style} style provided")
    
    return {"critique": response.content}


def revise_draft(state: WriterState) -> dict:
    """Revise while maintaining style."""
    draft = state["draft"]
    critique = state["critique"]
    topic = state["topic"]
    style = state["style"]
    style_instructions = get_style_prompt(style)
    revision_count = state["revision_count"]
    
    prompt = f"""Revise this draft about "{topic}" based on feedback.

{style_instructions}

Current draft:
{draft}

Feedback:
{critique}

Write an improved version that addresses the feedback while maintaining the {style} style."""
    
    response = llm.invoke(prompt)
    
    new_count = revision_count + 1
    print(f"✏️ Revision {new_count} complete")
    
    return {
        "draft": response.content,
        "revision_count": new_count
    }


def should_continue(state: WriterState) -> str:
    """Decide whether to continue revising."""
    if state["revision_count"] >= state["max_revisions"]:
        return "end"
    if "EXCELLENT" in state["critique"].upper():
        return "end"
    return "revise"


def create_graph():
    graph = StateGraph(WriterState)
    
    graph.add_node("write_draft", write_draft)
    graph.add_node("critique", critique_draft)
    graph.add_node("revise", revise_draft)
    
    graph.set_entry_point("write_draft")
    graph.add_edge("write_draft", "critique")
    graph.add_conditional_edges("critique", should_continue, {"revise": "revise", "end": END})
    graph.add_edge("revise", "critique")
    
    return graph.compile()


def main():
    print("=" * 50)
    print("🎨 Styled Self-Improving Writer")
    print("=" * 50)
    
    # Get topic
    topic = input("\n📝 Topic to write about:\n> ").strip()
    if not topic:
        topic = "The value of continuous learning"
    
    # Get style
    print("\n🎨 Available styles:")
    for style, info in STYLES.items():
        print(f"  - {style}: {info['description']}")
    
    style = input("\nChoose style (formal/casual/creative):\n> ").strip().lower()
    if style not in STYLES:
        style = "casual"
        print(f"(Using default: {style})")
    
    # Run for chosen style
    app = create_graph()
    
    result = app.invoke({
        "topic": topic,
        "style": style,
        "draft": "",
        "critique": "",
        "revision_count": 0,
        "max_revisions": 2
    })
    
    print("\n" + "=" * 50)
    print(f"📄 FINAL DRAFT ({style.upper()} STYLE):")
    print("=" * 50)
    print(result["draft"])
    
    # Bonus: Compare all three styles
    compare = input("\n\nCompare all three styles on same topic? (y/n): ").strip().lower()
    if compare == 'y':
        print("\n" + "=" * 50)
        print("🎨 STYLE COMPARISON")
        print("=" * 50)
        
        for style_name in STYLES:
            print(f"\n--- {style_name.upper()} ---")
            result = app.invoke({
                "topic": topic,
                "style": style_name,
                "draft": "",
                "critique": "",
                "revision_count": 0,
                "max_revisions": 1  # Just 1 revision for speed
            })
            print(result["draft"])


if __name__ == "__main__":
    main()


---
## Section 14.6 Solutions

### Exercise 14.6.1: Email Classifier

In [ ]:
# File: exercise_1_14_6_solution.py

"""Email classifier with multi-way routing.

Exercise 1 Solution: Build a graph that classifies incoming emails
and routes them to specialized handlers (5 categories).
"""

import os
from typing import TypedDict
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


# === STATE ===

class EmailState(TypedDict):
    email_subject: str        # Subject line
    email_body: str           # Full email content
    sender: str               # Who sent it
    category: str             # Classification result
    action_taken: str         # What we did with it
    extracted_info: dict      # Any info we pulled out


# === CLASSIFICATION NODE ===

def classify_email(state: EmailState) -> dict:
    """Classify the email into one of 5 categories.
    
    Uses the subject, body, and sender to determine the type.
    """
    subject = state["email_subject"]
    body = state["email_body"]
    sender = state["sender"]
    
    prompt = f"""Classify this email into exactly ONE category:

    From: {sender}
    Subject: {subject}
    Body: {body}

    Categories:
    - URGENT: Time-sensitive, needs immediate response, emergencies
    - MEETING: Calendar invites, scheduling, meeting requests
    - NEWSLETTER: Marketing, promotions, subscriptions
    - PERSONAL: From friends, family, personal contacts
    - SPAM: Unwanted, suspicious, phishing attempts

    Respond with only the category name."""
    
    response = llm.invoke(prompt)
    category = response.content.strip().upper()
    
    # Validate - default to NEWSLETTER if unrecognized
    valid = ["URGENT", "MEETING", "NEWSLETTER", "PERSONAL", "SPAM"]
    if category not in valid:
        category = "NEWSLETTER"
    
    print(f"📧 Classified as: {category}")
    return {"category": category}


# === ROUTING FUNCTION ===

def route_email(state: EmailState) -> str:
    """Route to the appropriate handler based on category."""
    category = state["category"]
    return f"handle_{category.lower()}"


# === HANDLER NODES ===

def handle_urgent(state: EmailState) -> dict:
    """Generate quick acknowledgment for urgent emails."""
    prompt = f"""Write a brief acknowledgment for this urgent email.
    From: {state['sender']}
    Subject: {state['email_subject']}
    
    Acknowledge receipt and promise quick response (2-3 sentences)."""
    
    response = llm.invoke(prompt)
    print("🚨 Urgent: Acknowledgment generated")
    
    return {
        "action_taken": "ACKNOWLEDGED",
        "extracted_info": {"response_draft": response.content}
    }


def handle_meeting(state: EmailState) -> dict:
    """Extract meeting details from the email."""
    prompt = f"""Extract meeting information from this email:
    Subject: {state['email_subject']}
    Body: {state['email_body']}
    
    Extract: date/time, participants, location/link, purpose.
    Format as a brief summary."""
    
    response = llm.invoke(prompt)
    print("📅 Meeting: Details extracted")
    
    return {
        "action_taken": "MEETING_EXTRACTED",
        "extracted_info": {"meeting_details": response.content}
    }


def handle_newsletter(state: EmailState) -> dict:
    """Archive newsletter emails."""
    print("📰 Newsletter: Archived")
    return {
        "action_taken": "ARCHIVED",
        "extracted_info": {"folder": "Newsletters", "source": state["sender"]}
    }


def handle_personal(state: EmailState) -> dict:
    """Flag personal emails for review."""
    print("👤 Personal: Flagged for review")
    return {
        "action_taken": "FLAGGED_PERSONAL",
        "extracted_info": {"flag": "Needs your attention"}
    }


def handle_spam(state: EmailState) -> dict:
    """Delete spam emails."""
    print("🗑️ Spam: Deleted")
    return {
        "action_taken": "DELETED",
        "extracted_info": {"blocked_sender": state["sender"]}
    }


# === GRAPH BUILDER ===

def create_email_graph():
    """Build the email classifier graph with 5-way routing."""
    graph = StateGraph(EmailState)
    
    # Add nodes
    graph.add_node("classify", classify_email)
    graph.add_node("handle_urgent", handle_urgent)
    graph.add_node("handle_meeting", handle_meeting)
    graph.add_node("handle_newsletter", handle_newsletter)
    graph.add_node("handle_personal", handle_personal)
    graph.add_node("handle_spam", handle_spam)
    
    # Entry and routing
    graph.set_entry_point("classify")
    
    graph.add_conditional_edges(
        "classify",
        route_email,
        {
            "handle_urgent": "handle_urgent",
            "handle_meeting": "handle_meeting",
            "handle_newsletter": "handle_newsletter",
            "handle_personal": "handle_personal",
            "handle_spam": "handle_spam"
        }
    )
    
    # All handlers end
    for handler in ["handle_urgent", "handle_meeting", "handle_newsletter", 
                    "handle_personal", "handle_spam"]:
        graph.add_edge(handler, END)
    
    return graph.compile()


# === MAIN ===

def main():
    app = create_email_graph()
    
    test_emails = [
        {"sender": "boss@company.com", "subject": "URGENT: Server down!", 
         "body": "Production crashed. Need help immediately!"},
        {"sender": "calendar@company.com", "subject": "Meeting: Q4 Planning",
         "body": "Friday at 2pm in Conference Room A."},
        {"sender": "deals@store.com", "subject": "50% OFF Today Only!",
         "body": "Our biggest sale of the year!"},
        {"sender": "mom@email.com", "subject": "Sunday dinner?",
         "body": "Are you coming for dinner? Love, Mom"},
        {"sender": "prince@scam.com", "subject": "You won $1,000,000!",
         "body": "Send your bank details to claim..."}
    ]
    
    print("=" * 60)
    print("📬 Email Classifier")
    print("=" * 60)
    
    for email in test_emails:
        print(f"\n📩 From: {email['sender']}")
        print(f"   Subject: {email['subject']}")
        print("-" * 40)
        
        result = app.invoke({
            "email_subject": email["subject"],
            "email_body": email["body"],
            "sender": email["sender"],
            "category": "",
            "action_taken": "",
            "extracted_info": {}
        })
        
        print(f"   Action: {result['action_taken']}")


if __name__ == "__main__":
    main()


### Exercise 14.6.2: Multi-Stage Interview

In [ ]:
# File: exercise_2_14_6_solution.py

"""Multi-stage interview bot with role-based branching.

Exercise 2 Solution: Create an interview bot with three stages:
- Stage 1: Basic info (name, background)
- Stage 2: Technical questions (different paths for engineer vs designer)
- Stage 3: Behavioral questions
"""

import os
from typing import TypedDict, Annotated
from operator import add
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


# === STATE ===

class InterviewState(TypedDict):
    candidate_name: str
    role: str                            # "engineer" or "designer"
    current_stage: int                   # 1, 2, or 3
    questions_asked: Annotated[list, add]
    responses: Annotated[list, add]
    stage_complete: bool
    interview_summary: str


# === STAGE 1: BASIC INFO ===

def stage1_basic_info(state: InterviewState) -> dict:
    """Stage 1: Gather basic information about the candidate."""
    name = state.get("candidate_name", "")
    
    if not name:
        question = "Welcome! What's your name?"
        # Simulated response (real app would get user input)
        return {
            "questions_asked": [question],
            "responses": ["Alex"],
            "candidate_name": "Alex",
            "stage_complete": False
        }
    else:
        question = f"Hi {name}! Tell me about your background."
        return {
            "questions_asked": [question],
            "responses": ["5 years experience..."],
            "stage_complete": True
        }


def check_stage1_complete(state: InterviewState) -> str:
    """Decide whether to continue stage 1 or advance."""
    if state["stage_complete"] and state.get("candidate_name"):
        return "advance_to_stage2"
    return "continue_stage1"


# === STAGE 2: ROLE-BASED TECHNICAL QUESTIONS ===

def advance_to_stage2(state: InterviewState) -> dict:
    """Transition to technical questions."""
    print("📈 Advancing to Stage 2: Technical Questions")
    return {"current_stage": 2, "stage_complete": False}


def route_by_role(state: InterviewState) -> str:
    """Route to role-specific technical questions."""
    role = state.get("role", "engineer").lower()
    if "design" in role:
        return "stage2_designer"
    return "stage2_engineer"


def stage2_engineer(state: InterviewState) -> dict:
    """Technical questions for engineering candidates."""
    questions = [
        "Describe a challenging technical problem you solved.",
        "What's your experience with system design?",
        "How do you approach debugging?"
    ]
    
    asked_count = len([q for q in state["questions_asked"] 
                       if "technical" in q.lower() or "system" in q.lower()])
    
    if asked_count < len(questions):
        q = questions[asked_count]
        print(f"🔧 Engineer Q: {q[:40]}...")
        return {
            "questions_asked": [q],
            "responses": [f"[Response to: {q[:20]}...]"],
            "stage_complete": asked_count >= len(questions) - 1
        }
    return {"stage_complete": True}


def stage2_designer(state: InterviewState) -> dict:
    """Technical questions for design candidates."""
    questions = [
        "Walk me through your design process.",
        "How do you incorporate user feedback?",
        "What prototyping tools do you use?"
    ]
    
    asked_count = len([q for q in state["questions_asked"] 
                       if "design" in q.lower() or "user" in q.lower()])
    
    if asked_count < len(questions):
        q = questions[asked_count]
        print(f"🎨 Designer Q: {q[:40]}...")
        return {
            "questions_asked": [q],
            "responses": [f"[Response to: {q[:20]}...]"],
            "stage_complete": asked_count >= len(questions) - 1
        }
    return {"stage_complete": True}


# === STAGE 3: BEHAVIORAL QUESTIONS ===

def advance_to_stage3(state: InterviewState) -> dict:
    """Transition to behavioral questions."""
    print("📈 Advancing to Stage 3: Behavioral Questions")
    return {"current_stage": 3, "stage_complete": False}


def stage3_behavioral(state: InterviewState) -> dict:
    """Behavioral questions for all candidates."""
    questions = [
        "Tell me about a time you worked with a difficult team member.",
        "Describe meeting a tight deadline.",
        "What motivates you?"
    ]
    
    asked_count = len([q for q in state["questions_asked"] 
                       if "tell me" in q.lower() or "describe" in q.lower()])
    
    if asked_count < len(questions):
        q = questions[asked_count]
        print(f"💭 Behavioral Q: {q[:40]}...")
        return {
            "questions_asked": [q],
            "responses": [f"[Response to: {q[:20]}...]"],
            "stage_complete": asked_count >= len(questions) - 1
        }
    return {"stage_complete": True}


# === SUMMARY ===

def generate_summary(state: InterviewState) -> dict:
    """Generate final interview summary."""
    summary = f"""
Interview Summary for {state['candidate_name']}
Role: {state['role']}
Questions Asked: {len(state['questions_asked'])}
Stages Completed: 3/3
"""
    print("📋 Interview complete!")
    return {"interview_summary": summary.strip()}


# === GRAPH BUILDER ===

def create_interview_graph():
    graph = StateGraph(InterviewState)
    
    # Add all nodes
    graph.add_node("stage1", stage1_basic_info)
    graph.add_node("advance_to_stage2", advance_to_stage2)
    graph.add_node("stage2_engineer", stage2_engineer)
    graph.add_node("stage2_designer", stage2_designer)
    graph.add_node("advance_to_stage3", advance_to_stage3)
    graph.add_node("stage3", stage3_behavioral)
    graph.add_node("summary", generate_summary)
    
    graph.set_entry_point("stage1")
    
    # Stage 1 loop or advance
    graph.add_conditional_edges("stage1", check_stage1_complete, {
        "continue_stage1": "stage1",
        "advance_to_stage2": "advance_to_stage2"
    })
    
    # Stage 2 role branching
    graph.add_conditional_edges("advance_to_stage2", route_by_role, {
        "stage2_engineer": "stage2_engineer",
        "stage2_designer": "stage2_designer"
    })
    
    # Stage 2 loops
    def check_stage2(state):
        return "advance_to_stage3" if state["stage_complete"] else "continue"
    
    graph.add_conditional_edges("stage2_engineer", check_stage2, {
        "continue": "stage2_engineer",
        "advance_to_stage3": "advance_to_stage3"
    })
    graph.add_conditional_edges("stage2_designer", check_stage2, {
        "continue": "stage2_designer", 
        "advance_to_stage3": "advance_to_stage3"
    })
    
    # Stage 3
    graph.add_edge("advance_to_stage3", "stage3")
    
    def check_stage3(state):
        return "summary" if state["stage_complete"] else "continue"
    
    graph.add_conditional_edges("stage3", check_stage3, {
        "continue": "stage3",
        "summary": "summary"
    })
    
    graph.add_edge("summary", END)
    
    return graph.compile()


# === MAIN ===

def main():
    app = create_interview_graph()
    
    print("=" * 60)
    print("🎤 Multi-Stage Interview Bot")
    print("=" * 60)
    
    # Test with engineer
    print("\n--- Engineering Candidate ---")
    result = app.invoke({
        "candidate_name": "",
        "role": "engineer",
        "current_stage": 1,
        "questions_asked": [],
        "responses": [],
        "stage_complete": False,
        "interview_summary": ""
    })
    print(result["interview_summary"])
    
    # Test with designer
    print("\n--- Design Candidate ---")
    result = app.invoke({
        "candidate_name": "",
        "role": "designer",
        "current_stage": 1,
        "questions_asked": [],
        "responses": [],
        "stage_complete": False,
        "interview_summary": ""
    })
    print(result["interview_summary"])


if __name__ == "__main__":
    main()


### Exercise 14.6.3: Retry with Backoff

In [ ]:
# File: exercise_3_14_6_solution.py

"""Research assistant with retry logic for low-quality results.

Exercise 3 Solution: Enhance a research assistant to handle poor-quality results:
- If search quality is LOW, retry with a modified query
- Track retries per search (max 2 retries)
- If still low after retries, move on to next search

Key insight: retry creates a mini-loop within the larger search loop.
"""

import os
import random
from typing import TypedDict, Annotated
from operator import add
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


# === STATE ===

class ResearchState(TypedDict):
    question: str
    search_queries: Annotated[list, add]
    findings: Annotated[list, add]
    current_query: str            # The active query
    current_quality: str          # HIGH, MEDIUM, LOW
    retry_count: int              # Retries for THIS search
    max_retries: int              # Limit per search (e.g., 2)
    search_count: int             # Total searches done
    max_searches: int             # Overall limit
    has_enough_info: bool         # Do we have enough?
    final_answer: str


# === NODES ===

def generate_search_query(state: ResearchState) -> dict:
    """Generate a search query based on the question and existing findings."""
    question = state["question"]
    findings = state.get("findings", [])
    
    if not findings:
        # First query - base it on the question
        prompt = f"Generate a concise search query for: {question}"
    else:
        # Subsequent queries - look for gaps
        prompt = f"""Question: {question}
        
        Already found: {len(findings)} results.
        
        Generate a NEW search query to find additional information."""
    
    response = llm.invoke(prompt)
    query = response.content.strip()
    
    print(f"🔍 New query: {query}")
    
    return {
        "current_query": query,
        "search_queries": [query],
        "search_count": state.get("search_count", 0) + 1,
        "retry_count": 0  # Reset retry count for new search
    }


def perform_search(state: ResearchState) -> dict:
    """Search and assess result quality."""
    query = state["current_query"]
    
    # Simulate search
    prompt = f"Simulate a search result for: {query}\nProvide a brief finding."
    response = llm.invoke(prompt)
    finding = response.content.strip()
    
    # Simulate quality (higher retry = better chance)
    # In real app, would actually assess the result
    quality_score = random.random() + (state["retry_count"] * 0.3)
    if quality_score > 0.7:
        quality = "HIGH"
    elif quality_score > 0.4:
        quality = "MEDIUM"
    else:
        quality = "LOW"
    
    print(f"📄 Quality: {quality}")
    
    return {
        "findings": [{"query": query, "result": finding, "quality": quality}],
        "current_quality": quality
    }


def route_after_search(state: ResearchState) -> str:
    """Decide: accept, retry, or move on."""
    quality = state["current_quality"]
    retry_count = state["retry_count"]
    max_retries = state["max_retries"]
    
    # High quality: accept
    if quality == "HIGH":
        print("✅ Good quality - accepting")
        return "evaluate"
    
    # Low quality with retries left: retry
    if quality == "LOW" and retry_count < max_retries:
        print(f"🔄 Low quality - retry {retry_count + 1}/{max_retries}")
        return "retry_search"
    
    # Medium or exhausted retries: accept and move on
    print("⚠️ Accepting (medium quality or max retries)")
    return "evaluate"


def retry_search(state: ResearchState) -> dict:
    """Modify query and increment retry count."""
    query = state["current_query"]
    
    prompt = f'The search "{query}" gave poor results. Suggest a better query.'
    response = llm.invoke(prompt)
    new_query = response.content.strip()
    
    print(f"🔄 Retry with: {new_query}")
    
    return {
        "current_query": new_query,
        "retry_count": state["retry_count"] + 1
    }


def evaluate_findings(state: ResearchState) -> dict:
    """Evaluate if we have enough information."""
    findings = state["findings"]
    question = state["question"]
    
    # Simple evaluation: do we have at least 2 high/medium quality findings?
    good_findings = [f for f in findings if f.get("quality") in ["HIGH", "MEDIUM"]]
    has_enough = len(good_findings) >= 2
    
    print(f"📊 Evaluation: {len(good_findings)} good findings, enough={has_enough}")
    
    return {"has_enough_info": has_enough}


def route_after_evaluate(state: ResearchState) -> str:
    """Decide: search more or synthesize answer."""
    has_enough = state["has_enough_info"]
    search_count = state["search_count"]
    max_searches = state["max_searches"]
    
    # Safety valve: stop at max searches
    if search_count >= max_searches:
        print(f"🛑 Max searches ({max_searches}) reached")
        return "synthesize"
    
    # Enough info: done
    if has_enough:
        print("✅ Enough info gathered")
        return "synthesize"
    
    # Otherwise, keep searching
    return "search_more"


def synthesize_answer(state: ResearchState) -> dict:
    """Synthesize final answer from findings."""
    findings = state["findings"]
    question = state["question"]
    
    findings_text = "\n".join([f"- {f['result']}" for f in findings])
    
    prompt = f"""Based on these findings, answer the question.
    
    Question: {question}
    
    Findings:
    {findings_text}
    
    Provide a concise answer."""
    
    response = llm.invoke(prompt)
    
    print("📝 Answer synthesized")
    
    return {"final_answer": response.content}


# === GRAPH BUILDER ===

def create_research_graph():
    """Build the research assistant graph with retry logic.
    
    Two levels of looping:
    1. Outer loop: search → evaluate → maybe search again
    2. Inner loop: search → quality check → maybe retry same search
    """
    graph = StateGraph(ResearchState)
    
    graph.add_node("generate_query", generate_search_query)
    graph.add_node("search", perform_search)
    graph.add_node("retry_search", retry_search)
    graph.add_node("evaluate", evaluate_findings)
    graph.add_node("synthesize", synthesize_answer)
    
    graph.set_entry_point("generate_query")
    
    graph.add_edge("generate_query", "search")
    
    # After search: accept, retry, or evaluate
    graph.add_conditional_edges("search", route_after_search, {
        "retry_search": "retry_search",
        "evaluate": "evaluate"
    })
    
    # Retry loops back to search
    graph.add_edge("retry_search", "search")
    
    # After evaluate: more searching or synthesize
    graph.add_conditional_edges("evaluate", route_after_evaluate, {
        "search_more": "generate_query",
        "synthesize": "synthesize"
    })
    
    graph.add_edge("synthesize", END)
    
    return graph.compile()


# === MAIN ===

def main():
    app = create_research_graph()
    
    print("=" * 60)
    print("🔬 Research Assistant with Retry Logic")
    print("=" * 60)
    
    result = app.invoke({
        "question": "What are the main benefits of using TypeScript over JavaScript?",
        "search_queries": [],
        "findings": [],
        "current_query": "",
        "current_quality": "",
        "retry_count": 0,
        "max_retries": 2,
        "search_count": 0,
        "max_searches": 4,
        "has_enough_info": False,
        "final_answer": ""
    })
    
    print("\n" + "=" * 60)
    print("📋 Final Answer:")
    print("=" * 60)
    print(result["final_answer"])
    print(f"\n📊 Stats: {result['search_count']} searches, {len(result['findings'])} findings")


if __name__ == "__main__":
    main()


---
## Section 14.7 Solutions

### Exercise 14.7.1: Add Debugging to the Ticket Router

In [ ]:
# File: exercise_1_14_7_solution.py

"""Ticket router with full debugging capabilities.

Exercise 1 Solution: Add comprehensive debugging to the ticket router:
- Debug output for every node
- State tracking
- Loop counter safety valve
- Graph visualization at startup
"""

import os
from typing import TypedDict
from datetime import datetime
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Debug flag - set to False in production
DEBUG = True


def debug_print(*args, **kwargs):
    """Print only when DEBUG is True."""
    if DEBUG:
        print(*args, **kwargs)


# === STATE (with safety counter) ===

class TicketState(TypedDict):
    ticket_text: str
    category: str
    priority: str
    response: str
    needs_human: bool
    # Safety counter (even though this graph shouldn't loop)
    node_visits: int


# === STATE TRACKER ===

class StateTracker:
    """Track state changes throughout execution."""
    
    def __init__(self):
        self.history = []
    
    def capture(self, node_name: str, state: dict, updates: dict = None):
        import copy
        self.history.append({
            "timestamp": datetime.now().isoformat(),
            "node": node_name,
            "state": copy.deepcopy(dict(state)),
            "updates": copy.deepcopy(updates) if updates else None
        })
    
    def print_summary(self):
        print("\n" + "=" * 60)
        print("📜 EXECUTION TRACE")
        print("=" * 60)
        
        for i, entry in enumerate(self.history):
            print(f"\nStep {i+1}: {entry['node']}")
            if entry['updates']:
                for k, v in entry['updates'].items():
                    print(f"  → {k}: {str(v)[:50]}")


# Global tracker
tracker = StateTracker()


# === NODES (with debug output) ===

def classify_ticket(state: TicketState) -> dict:
    """Classify with full debug output."""
    debug_print(f"\n{'='*50}")
    debug_print(f"🔵 ENTERING: classify_ticket")
    debug_print(f"   Ticket: {state['ticket_text'][:40]}...")
    
    # Safety check
    visits = state.get("node_visits", 0) + 1
    if visits > 10:
        raise Exception("Safety limit: too many node visits!")
    
    ticket = state["ticket_text"]
    
    prompt = f"""Classify this support ticket.
    
    Ticket: {ticket}
    
    Categories: BILLING, TECHNICAL, ACCOUNT, GENERAL
    Priority: HIGH, MEDIUM, LOW
    
    Format:
    CATEGORY: <category>
    PRIORITY: <priority>"""
    
    response = llm.invoke(prompt)
    content = response.content.upper()
    
    # Parse with defaults
    category = "GENERAL"
    for cat in ["BILLING", "TECHNICAL", "ACCOUNT"]:
        if cat in content:
            category = cat
            break
    
    priority = "MEDIUM"
    for pri in ["HIGH", "LOW"]:
        if pri in content:
            priority = pri
            break
    
    updates = {
        "category": category,
        "priority": priority,
        "node_visits": visits
    }
    
    debug_print(f"   Result: {category} ({priority})")
    tracker.capture("classify_ticket", state, updates)
    
    return updates


def route_by_category(state: TicketState) -> str:
    """Route with debug output."""
    category = state["category"]
    priority = state["priority"]
    
    if priority == "HIGH":
        decision = "escalate"
    else:
        routes = {
            "BILLING": "handle_billing",
            "TECHNICAL": "handle_technical", 
            "ACCOUNT": "handle_account",
            "GENERAL": "handle_general"
        }
        decision = routes.get(category, "handle_general")
    
    debug_print(f"🔀 ROUTING: {decision}")
    debug_print(f"   (category={category}, priority={priority})")
    
    return decision


# === HANDLER FACTORY ===

def make_handler(name: str, emoji: str, specialty: str):
    """Factory to create debug-wrapped handlers."""
    
    def handler(state: TicketState) -> dict:
        debug_print(f"\n{'='*50}")
        debug_print(f"🔵 ENTERING: {name}")
        
        visits = state.get("node_visits", 0) + 1
        
        prompt = f"""You are a {specialty} specialist. Help with:
        {state['ticket_text']}
        Keep response brief (2-3 sentences)."""
        
        response = llm.invoke(prompt)
        
        updates = {
            "response": response.content,
            "needs_human": False,
            "node_visits": visits
        }
        
        debug_print(f"{emoji} Response generated")
        tracker.capture(name, state, updates)
        
        return updates
    
    return handler


# Create handlers
handle_billing = make_handler("handle_billing", "💳", "billing support")
handle_technical = make_handler("handle_technical", "🔧", "technical support")
handle_account = make_handler("handle_account", "👤", "account support")
handle_general = make_handler("handle_general", "📧", "general support")


def escalate_ticket(state: TicketState) -> dict:
    debug_print(f"\n{'='*50}")
    debug_print(f"🔵 ENTERING: escalate_ticket")
    
    updates = {
        "response": "Escalated to senior agent. Response within 1 hour.",
        "needs_human": True,
        "node_visits": state.get("node_visits", 0) + 1
    }
    
    debug_print("🚨 Ticket escalated!")
    tracker.capture("escalate_ticket", state, updates)
    
    return updates


# === GRAPH (with visualization) ===

def create_debug_router():
    """Build graph and show visualization."""
    graph = StateGraph(TicketState)
    
    graph.add_node("classify", classify_ticket)
    graph.add_node("handle_billing", handle_billing)
    graph.add_node("handle_technical", handle_technical)
    graph.add_node("handle_account", handle_account)
    graph.add_node("handle_general", handle_general)
    graph.add_node("escalate", escalate_ticket)
    
    graph.set_entry_point("classify")
    
    graph.add_conditional_edges(
        "classify",
        route_by_category,
        {
            "handle_billing": "handle_billing",
            "handle_technical": "handle_technical",
            "handle_account": "handle_account",
            "handle_general": "handle_general",
            "escalate": "escalate"
        }
    )
    
    for handler in ["handle_billing", "handle_technical", 
                    "handle_account", "handle_general", "escalate"]:
        graph.add_edge(handler, END)
    
    app = graph.compile()
    
    # Show graph structure at startup
    if DEBUG:
        print("\n📊 GRAPH STRUCTURE")
        print("-" * 40)
        print(app.get_graph().draw_mermaid())
        print("-" * 40)
    
    return app


# === MAIN ===

def main():
    app = create_debug_router()
    
    test_tickets = [
        "I was charged twice!",
        "App keeps crashing",
        "THIS IS UNACCEPTABLE! FIX IT NOW!"
    ]
    
    for ticket in test_tickets:
        tracker.history.clear()  # Reset for each ticket
        
        print(f"\n{'='*60}")
        print(f"📩 Processing: {ticket}")
        print("=" * 60)
        
        result = app.invoke({
            "ticket_text": ticket,
            "category": "",
            "priority": "",
            "response": "",
            "needs_human": False,
            "node_visits": 0
        })
        
        # Print execution trace
        tracker.print_summary()
        
        print(f"\n✅ Final: {result['category']} → {result['response'][:50]}...")


if __name__ == "__main__":
    main()


### Exercise 14.7.2: Find the Bug

In [ ]:
# File: exercise_2_14_7_solution.py

"""Fixed version of the buggy graph with debugging.

Exercise 2 Solution: Find and fix all the bugs in the provided code.

Bugs found and fixed:
1. results: list → results: Annotated[list, add] (so results accumulate)
2. state["search_count"] → state.get("search_count", 0) (KeyError fix)
3. Routing function returns must match mapping keys exactly
"""

from typing import TypedDict, Annotated
from operator import add
from langgraph.graph import StateGraph, END


# BUG 1 FIX: Use Annotated[list, add] so results accumulate
class FixedState(TypedDict):
    query: str
    results: Annotated[list, add]  # FIXED: Was just 'list'
    search_count: int
    max_searches: int


def search(state: FixedState) -> dict:
    query = state["query"]
    
    # BUG 2 FIX: Use .get() with default value
    count = state.get("search_count", 0)  # FIXED: Was state["search_count"]
    
    print(f"🔍 Search #{count + 1}: {query}")
    
    result = f"Result for: {query}"
    
    return {
        "results": [result],  # This now accumulates thanks to Annotated
        "search_count": count + 1
    }


def should_continue(state: FixedState) -> str:
    """Fixed routing function with proper return values."""
    current = state.get("search_count", 0)
    maximum = state.get("max_searches", 3)
    
    print(f"🔀 Checking: {current} < {maximum}?")
    
    if current < maximum:
        # BUG 3 FIX: Return value must match mapping keys
        return "continue"  # FIXED: Was "search" which didn't match mapping
    return "done"  # FIXED: Was "end" which didn't match mapping


def create_fixed_graph():
    graph = StateGraph(FixedState)
    
    graph.add_node("search", search)
    graph.set_entry_point("search")
    
    # BUG 3 FIX: Mapping keys must match what routing function returns
    graph.add_conditional_edges(
        "search",
        should_continue,
        {
            "continue": "search",  # Loops back
            "done": END            # Exits
        }
    )
    
    return graph.compile()


def main():
    app = create_fixed_graph()
    
    print("🐛 Running fixed graph...")
    print("-" * 40)
    
    result = app.invoke({
        "query": "LangGraph tutorials",
        "results": [],          # Initialize empty list
        "search_count": 0,      # Initialize counter
        "max_searches": 3
    })
    
    print("-" * 40)
    print(f"✅ Total searches: {result['search_count']}")
    print(f"✅ Results collected: {len(result['results'])}")
    for i, r in enumerate(result['results'], 1):
        print(f"   {i}. {r}")


if __name__ == "__main__":
    main()


# === SUMMARY OF BUGS ===
"""
Bug 1: results: list should be results: Annotated[list, add]
  - Without Annotated, list gets replaced each time
  - With Annotated[list, add], lists accumulate across iterations

Bug 2: state["search_count"] throws KeyError if not initialized
  - Fixed with state.get("search_count", 0)
  - Always use .get() with defaults for safety

Bug 3: Routing function returned "search" and "end", but mapping had "continue" and "done"
  - The return values must EXACTLY match the mapping keys
  - This is a very common bug - always double-check your mapping!
"""


### Exercise 14.7.3: Build a Debug Dashboard

In [ ]:
# File: exercise_3_14_7_solution.py

"""Debug dashboard for analyzing LangGraph executions.

Exercise 3 Solution: Build a debug dashboard that produces:
- Total nodes visited
- Time spent
- State changes for each field
- Fields that never changed
- Routing decisions made
"""

from datetime import datetime
from collections import defaultdict


class DebugDashboard:
    """Comprehensive execution analysis dashboard."""
    
    def __init__(self):
        self.executions = []
        self.current_execution = None
    
    def start_execution(self, name: str = None):
        """Start tracking a new execution."""
        self.current_execution = {
            "name": name or f"Execution_{len(self.executions) + 1}",
            "started": datetime.now(),
            "ended": None,
            "steps": [],
            "routing_decisions": [],
            "initial_state": None,
            "final_state": None
        }
    
    def record_step(self, node_name: str, state_before: dict, updates: dict):
        """Record a single step in the execution."""
        if not self.current_execution:
            self.start_execution()
        
        step = {
            "timestamp": datetime.now(),
            "node": node_name,
            "state_before": dict(state_before),
            "updates": dict(updates) if updates else {}
        }
        self.current_execution["steps"].append(step)
        
        # Track initial state
        if self.current_execution["initial_state"] is None:
            self.current_execution["initial_state"] = dict(state_before)
    
    def record_routing(self, router_name: str, decision: str, reason: str = None):
        """Record a routing decision."""
        if self.current_execution:
            self.current_execution["routing_decisions"].append({
                "timestamp": datetime.now(),
                "router": router_name,
                "decision": decision,
                "reason": reason
            })
    
    def end_execution(self, final_state: dict):
        """End the current execution."""
        if self.current_execution:
            self.current_execution["ended"] = datetime.now()
            self.current_execution["final_state"] = dict(final_state)
            self.executions.append(self.current_execution)
            self.current_execution = None
    
    def generate_report(self, execution_index: int = -1) -> str:
        """Generate a comprehensive report for an execution."""
        if not self.executions:
            return "No executions recorded."
        
        exec_data = self.executions[execution_index]
        
        lines = []
        lines.append("=" * 60)
        lines.append(f"📊 DEBUG REPORT: {exec_data['name']}")
        lines.append("=" * 60)
        
        # Timing
        duration = (exec_data['ended'] - exec_data['started']).total_seconds()
        lines.append(f"\n⏱️  TIMING")
        lines.append(f"   Started: {exec_data['started'].strftime('%H:%M:%S.%f')[:-3]}")
        lines.append(f"   Ended: {exec_data['ended'].strftime('%H:%M:%S.%f')[:-3]}")
        lines.append(f"   Duration: {duration:.3f} seconds")
        
        # Node visits
        lines.append(f"\n📍 NODE VISITS ({len(exec_data['steps'])} total)")
        node_counts = defaultdict(int)
        for step in exec_data['steps']:
            node_counts[step['node']] += 1
        for node, count in node_counts.items():
            marker = "⚠️ " if count > 1 else "   "
            lines.append(f"{marker}{node}: {count} visit(s)")
        
        # Routing decisions
        lines.append(f"\n🔀 ROUTING DECISIONS ({len(exec_data['routing_decisions'])})")
        for rd in exec_data['routing_decisions']:
            reason = f" ({rd['reason']})" if rd['reason'] else ""
            lines.append(f"   {rd['router']} → {rd['decision']}{reason}")
        
        # State changes
        lines.append(f"\n📝 STATE CHANGES")
        all_fields = set()
        for step in exec_data['steps']:
            all_fields.update(step['state_before'].keys())
            all_fields.update(step['updates'].keys())
        
        initial = exec_data['initial_state'] or {}
        final = exec_data['final_state'] or {}
        
        for field in sorted(all_fields):
            initial_val = initial.get(field, "<not set>")
            final_val = final.get(field, "<not set>")
            
            # Truncate long values
            iv_str = str(initial_val)[:30]
            fv_str = str(final_val)[:30]
            
            if initial_val == final_val:
                lines.append(f"   {field}: {fv_str} (unchanged)")
            else:
                lines.append(f"   {field}: {iv_str} → {fv_str}")
        
        # Fields that never changed
        unchanged = []
        for field in all_fields:
            if initial.get(field) == final.get(field) and field in initial:
                unchanged.append(field)
        
        if unchanged:
            lines.append(f"\n⚠️  NEVER CHANGED: {', '.join(unchanged)}")
        
        lines.append("\n" + "=" * 60)
        
        return "\n".join(lines)


# === EXAMPLE USAGE ===

def example_usage():
    """Demonstrate the dashboard."""
    dashboard = DebugDashboard()
    
    # Simulate an execution
    dashboard.start_execution("Test Run")
    
    state = {"query": "test", "results": [], "count": 0}
    
    dashboard.record_step("search", state, {"results": ["r1"], "count": 1})
    dashboard.record_routing("should_continue", "continue", "count < max")
    
    state = {"query": "test", "results": ["r1"], "count": 1}
    dashboard.record_step("search", state, {"results": ["r2"], "count": 2})
    dashboard.record_routing("should_continue", "done", "count >= max")
    
    final = {"query": "test", "results": ["r1", "r2"], "count": 2}
    dashboard.end_execution(final)
    
    print(dashboard.generate_report())


if __name__ == "__main__":
    example_usage()


# === HOW TO USE IN YOUR GRAPH ===
"""
from debug_dashboard import DebugDashboard

dashboard = DebugDashboard()

def my_node(state: MyState) -> dict:
    # ... your logic ...
    updates = {"field": "value"}
    
    dashboard.record_step("my_node", state, updates)
    return updates

def my_router(state: MyState) -> str:
    decision = "some_path"
    dashboard.record_routing("my_router", decision, f"score={state.get('score')}")
    return decision

# In main:
dashboard.start_execution("My Test")
result = app.invoke(initial_state)
dashboard.end_execution(result)
print(dashboard.generate_report())
"""


---
## Next Steps

Return to **Chapter 15: Next Topic**